In [7]:
# !pip install ta

In [8]:
import numpy as np
import pandas as pd

import yfinance as yf
import ta

In [3]:
stockCode = input("Enter ticker: ")

Enter ticker:  PLTR


In [40]:
ticker = yf.Ticker(stockCode)
price_hist = ticker.history(period="max", interval="1d")
price_hist.shape

(736, 7)

In [41]:
train_ratio = 0.6
test_ratio = 0.2

l = len(price_hist)
train_idx = round(l * train_ratio)
test_idx = round(l * (train_ratio + test_ratio))

print(train_idx, test_idx)

442 589


In [69]:
price_hist = price_hist.drop(columns=["Dividends", "Stock Splits"])

train_df = price_hist.iloc[:train_idx, :]
test_df = price_hist.iloc[train_idx:test_idx, :]
valid_df = price_hist.iloc[test_idx:, :]

print(train_df.shape, test_df.shape, valid_df.shape)

(442, 5) (147, 5) (147, 5)


In [72]:
X_train = ta.add_momentum_ta(df=train_df.copy(), high="High", low="Low", close="Close", volume="Volume")
X_train["Target"] = X_train["Close"].shift(1) - X_train["Close"]
X_train = X_train.dropna()

In [73]:
X_train.corr()

,Open,High,Low,Close,Volume,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,...,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,Target
Open,1.000000,0.992041,0.994338,0.989471,0.194254,0.493526,-0.074927,-0.040907,-0.025211,0.603253,...,0.454251,0.323349,0.607590,0.632309,-0.031657,-0.015999,-0.038782,0.027036,0.970981,0.026061
High,0.992041,1.000000,0.987919,0.995136,0.246519,0.524513,-0.047226,-0.027363,-0.015831,0.616386,...,0.469832,0.367725,0.625425,0.643101,-0.004563,0.013047,-0.030268,0.067316,0.963265,-0.055992
Low,0.994338,0.987919,1.000000,0.993164,0.158440,0.499972,-0.042475,-0.024875,-0.015290,0.593626,...,0.433405,0.312375,0.590553,0.614875,-0.031836,-0.045096,-0.056731,0.000285,0.971331,-0.026869
Close,0.989471,0.995136,0.993164,1.000000,0.212746,0.529906,-0.021212,-0.016485,-0.011600,0.608304,...,0.448523,0.354788,0.610910,0.629892,-0.010651,-0.014571,-0.045558,0.039403,0.965886,-0.098903
Volume,0.194254,0.246519,0.158440,0.212746,1.000000,0.126702,-0.109037,-0.126264,-0.126080,0.182619,...,0.124240,0.137771,0.185421,0.216073,-0.093002,0.636218,0.357904,0.634822,0.236439,-0.103843
momentum_rsi,0.493526,0.524513,0.499972,0.529906,0.126702,1.000000,0.445308,0.467361,0.467908,0.878535,...,0.765754,0.811307,0.801649,0.674386,0.534848,-0.052409,-0.116096,0.072759,0.346428,-0.317294
momentum_stoch_rsi,-0.074927,-0.047226,-0.042475,-0.021212,-0.109037,0.445308,1.000000,0.903688,0.778724,0.005775,...,-0.015711,0.422597,-0.061896,-0.248000,0.665345,-0.128603,-0.103960,-0.082675,-0.180273,-0.419436
momentum_stoch_rsi_k,-0.040907,-0.027363,-0.024875,-0.016485,-0.126264,0.467361,0.903688,1.000000,0.951835,0.088424,...,0.086234,0.481744,0.009588,-0.207280,0.783062,-0.135990,-0.126895,-0.062932,-0.167851,-0.176079
momentum_stoch_rsi_d,-0.025211,-0.015831,-0.015290,-0.011600,-0.126080,0.467908,0.778724,0.951835,1.000000,0.160180,...,0.179353,0.510769,0.074449,-0.151809,0.823064,-0.128847,-0.139077,-0.032414,-0.146818,-0.080588
momentum_tsi,0.603253,0.616386,0.593626,0.608304,0.182619,0.878535,0.005775,0.088424,0.160180,1.000000,...,0.886304,0.693019,0.941472,0.895047,0.256527,-0.014718,-0.092374,0.106731,0.497296,-0.070715


In [74]:
y_train = X_train["Target"]
X_train = X_train.drop(columns=["Target"])

print(f"X shape: {X_train.shape}, y shape:{y_train.shape}")

X shape: (405, 23), y shape:(405,)


In [78]:
from sklearn.tree import DecisionTreeRegressor

np.random.seed(123)

model = DecisionTreeRegressor()
model = model.fit(X_train, y_train)
model

DecisionTreeRegressor()